In [9]:
import pandas as pd
import re

# Load your DataFrame
df = pd.read_csv('clean_data.csv')

In [10]:
df

,Unnamed: 0,message_id,from_id,date,message_text
0,0,13059,NaN,2024-09-28 14:06:43+00:00,High pressure water Gun nozzle\r\n\r\n መኪና ለ...
1,1,13058,NaN,2024-09-28 13:26:43+00:00,ገብቷል ውስን ፍሬ ይፍጠኑ\r\n\r\n ቤቶ ሁነው የተለያዩ ስራዎችን ...
2,2,13057,NaN,2024-09-28 09:15:17+00:00,ወርቃማው የሲኒ ረከቦት\r\n\r\n4 in 1 የጀበና ማስቀመጫ፣ የስኳን...
3,3,13056,NaN,2024-09-27 11:16:28+00:00,ወርቃማው የሲኒ ረከቦት\r\n\r\n4 in 1 የጀበና ማስቀመጫ፣ የስኳን...
4,4,13055,NaN,2024-09-26 10:53:59+00:00,ወርቃማው የሲኒ ረከቦት\r\n\r\n4 in 1 የጀበና ማስቀመጫ፣ የስኳን...
...,...,...,...,...,...
4388,6260,6415,NaN,2021-10-12 11:55:20+00:00,Smiling foot protector \r\n390 birr \r\nE020 \...
4389,6261,6414,NaN,2021-10-12 11:55:20+00:00,ለቤቶ ለስራ ቦታዎ ምርጥ የሴራሚክ መጥረጊያ ማሽኝ\r\n\r\n\r\nTe...
4390,6262,6413,NaN,2021-10-12 11:55:20+00:00,Kitchen Oil proof Stickers\r\nለኪችንዎ ግርማሞገስ የሚያላብስ
4391,6264,6411,NaN,2021-10-12 11:55:20+00:00,Kitchen Oil proof Stickers\r\nለኪችንዎ ግርማሞገስ የሚያ...


In [11]:

# List of B-PRODUCT items
b_product_items = ["መኪና", "ፍሬ", "የሲኒ", "የጀበና", "ማስቀመጫ", "የስኳን","ኦርቢት","ቲቪ","ግሪል አሩስቶ መስሪያ", "ለምሳቃ", "የፀጉር ማንሻ","አረቢያን መጅሊሶች",
                    "ማግ","የወጥ ማቅረቢያ","አጎበር" ,"ሳቺ", "ኦቮን","የልብስ የጫማ ማጠቢያ", "ሲሊንደር", "ሚንዳ","ኤልጂ" ,"ኢኮሳን", "የከሰል ምድጃ", "ሳንፎርድ", "የቂጣ መስሪያ","ዲጂታል ሚዛን",
                    "የሚዞር ኮስሞቲክስ መደርደሪያ","ቶሺባ","ማይክሮ ዌቭ","ተሽከርካሪ የቅመማቅመም ማስቀመጫ", "መመገቢያ ሳህን", "የቡና መፍጫ", " የስጋ መፍጫ", "ጭልፋዎች", "ሴራሚክ ድስቶች",
                    "ኩከር ድስት","የእጣን","የዘይት መጭመቂያ ማሽን","የቅመማቅመም መፍጫ","የዕቃ ማድረቂያ"]
# List of I-LOC items
i_loc_items = ['ቤተል አደባባይ', 'ለቡ', 'ኮልፌ አጠናተራ', 'ኑረዲን ሱፐር ማርኬት', 'አዲስ አበባ', 'ሱ,ቁ 095 ወይም 046', 'እፎይታ የገበያ ማዕከል','ሳራ ሞል']

# List of B-LOC items
b_loc_items = ['አዲስ አበባ', 'ኮልፌ አጠናተራ']



def label_message_utf8_with_birr(message):
    tokens = re.findall(r'\S+', message)  # Tokenize while considering non-ASCII characters
    labeled_tokens = []
    previous_tokens = []

    for i, token in enumerate(tokens):
        # Check if token is a price (e.g., 500 ETB, $100, or ብር)
        if re.match(r'^\d{10,}$', token):
            labeled_tokens.append(f"{token} O")  # Label as O for "other" or outside of any entity
        elif re.match(r'^\d+(\.\d{1,2})?$', token):
            # Check if the next token is "ብር", "ETB", or "etb" to label as B-PRICE
            if i + 1 < len(tokens) and tokens[i + 1] in ["ብር", "ዋጋ", "birr", 'ብር', 'ETB', 'etb']:
                labeled_tokens.append(f"{token} B-PRICE")
            else:
                labeled_tokens.append(f"{token} O")
        elif token in ['ብር', 'ETB', 'etb']:
            labeled_tokens.append(f"{token} I-PRICE")
            # Check if the previous token is a number to label as B-PRICE
            if i > 0 and re.match(r'^\d+(\.\d{1,2})?$', tokens[i - 1]):
                labeled_tokens[-2] = f"{tokens[i - 1]} B-PRICE"
        # Check if token is in the B-PRODUCT list
        elif token in b_product_items:
            # Label the previous two tokens as I-PRODUCT if they exist
            if len(previous_tokens) >= 2:
                labeled_tokens[-2] = f"{previous_tokens[-2]} I-PRODUCT"
                labeled_tokens[-1] = f"{previous_tokens[-1]} I-PRODUCT"
            elif len(previous_tokens) == 1:
                labeled_tokens[-1] = f"{previous_tokens[-1]} I-PRODUCT"
            labeled_tokens.append(f"{token} B-PRODUCT")
        # Check if token is in the I-LOC list
        elif token in i_loc_items:
            labeled_tokens.append(f"{token} I-LOC")
        # Check if token is in the B-LOC list
        elif token in b_loc_items:
            labeled_tokens.append(f"{token} B-LOC")
        # Assume other tokens are part of a product name (this can be refined)
        else:
            labeled_tokens.append(f"{token} O")

        # Keep track of the previous tokens
        previous_tokens.append(token)
        if len(previous_tokens) > 2:
            previous_tokens.pop(0)

    return "\n".join(labeled_tokens)


# Apply the updated function to the non-null messages
df['Labeled_Message'] = df['message_text'].apply(label_message_utf8_with_birr)

# Display the updated DataFrame
df.head()

# Save the updated labeled dataset to a file in CoNLL format
labeled_data_birr_path = 'Labeled_nejash_product_price_location.txt-'
with open(labeled_data_birr_path, 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        f.write(f"{row['Labeled_Message']}\n\n")

In [12]:
df['Labeled_Message'].head

<bound method NDFrame.head of 0       High O\npressure O\nwater O\nGun I-PRODUCT\nno...
1       ገብቷል I-PRODUCT\nውስን I-PRODUCT\nፍሬ B-PRODUCT\nይ...
2       ወርቃማው I-PRODUCT\nየሲኒ B-PRODUCT\nረከቦት O\n4 O\ni...
3       ወርቃማው I-PRODUCT\nየሲኒ B-PRODUCT\nረከቦት O\n4 O\ni...
4       ወርቃማው I-PRODUCT\nየሲኒ B-PRODUCT\nረከቦት O\n4 O\ni...
                              ...                        
4388    Smiling O\nfoot O\nprotector O\n390 B-PRICE\nb...
4389    ለቤቶ O\nለስራ O\nቦታዎ O\nምርጥ O\nየሴራሚክ O\nመጥረጊያ O\n...
4390    Kitchen O\nOil O\nproof O\nStickers O\nለኪችንዎ O...
4391    Kitchen O\nOil O\nproof O\nStickers O\nለኪችንዎ O...
4392    የልጆች O\nመማሪያ O\nታብሌት O\nTelegram O\n: O\nhttps...
Name: Labeled_Message, Length: 4393, dtype: object>